In [77]:
msr = {}
with open('data/clf/msr_positive_pairs.txt') as f:
    for t in f.readlines():
        r, n1, n2 = t.split()
        msr[(r, n1)] = n2

files = ['rlog/small_sample_for_recall.txt_leave_file_list.out',
        'rlog/small_sample_for_recall.txt_leave_code.out',
        'rlog/small_sample_for_recall.txt_leave_text.out',
        'rlog/small_sample_for_recall.txt_leave_location.out',
        'rlog/small_sample_for_recall.txt_leave_pattern.out',
        'rlog/small_sample_for_recall.txt_new.out']

threshold = 0

def check(file):
    # threshold = 0.6257216989

    num, tot = 0, 0

    num2, num3 = 0, 0

    w = []
    with open(file) as f:
        for t in f.readlines():
            tot += 1

            ts = t.split(':')
            r, n2 = ts[0].strip().split()
            op = ts[1].strip()[2:-2].split('), (')

            n1, v = op[0].split(',')

            v = float(v)

            w.append(v)

            if v < threshold:
                continue


            if n1 == msr.get((r, n2), ''):
                num2 += 1

            for obj in op[:5]:
                n1, v = obj.split(',')
                if n1 == msr.get((r, n2), ''):
                    num3 += 1
                    break

            num += 1

    #print(num, tot, 1.0 * num / tot)

    #print(file, 1.0 * num2 / tot, num2, tot)
    # print(1.0 * num3 / tot, num3, tot)

    #print(len(w), '->', 50)
    w = sorted(w, reverse=True)
    # print(file, ':', w[39])
    tw = file.split('_')[-1].split('.out')[0]
    if tw == 'list':
        tw = 'file_list'
    tw = '\'' + tw + '\''
    print(tw, ':', w[46 - 1], ',')
    

for file in files:
    check(file)




'file_list' : 0.6019025362680538 ,
'code' : 0.5795580260828507 ,
'text' : 0.5980482690330458 ,
'location' : 0.6000522318058933 ,
'pattern' : 0.5930564554337777 ,
'new' : 0.5961868169890219 ,


In [ ]:
from git import *

way_select = ['file_list', 'code', 'text', 'location', 'pattern', 'new']


def work(way):
    file = 'rlog/sample_small_data.txt_leave_%s.out' % way

    if way == 'new':
        file = 'rlog/sample_small_data.txt_new.out'

    outfile = 'rlog/webpage_%s.html' % way
    with open(outfile, 'w') as outf:
        pass

    
    '''
    threshold_select =  { #% 20%
    'file_list' : 0.6081185134217753,
    'code' : 0.5815402870741628,
    'text' : 0.6065632181226785,
    'location' : 0.6069270229613258,
    'pattern' : 0.5994309119726756,
    'new' : 0.6023691773882736,
    }
    '''
    
    '''
    threshold_select =  { # 15%
    'file_list' : 0.6233914080341519 ,
    'code' : 0.5891530735030057 ,
    'text' : 0.6440541699084077 ,
    'location' : 0.6233203523695693 ,
    'pattern' : 0.6173069060241599 ,
    'new' : 0.6140082744041935 ,
    }
    '''
    
    threshold_select = { # %23
    'file_list' : 0.6007059491295231 ,
    'code' : 0.5784769795625462 ,
    'text' : 0.5907405621707191 ,
    'location' : 0.5998763656071796 ,
    'pattern' : 0.5906280521823456 ,
    'new' : 0.5955144918569139 ,
    }
    
    
    p_tot = 0
    p_num = 0

    tot = 0
    threshold = threshold_select[way]
        
    label = {}
    with open('rlog/labeled.txt') as f:
        for t in f.readlines():
            r, n2, n1, v, status = t.split()
            if 'Unknown' in status:
                continue
            label[(r, n2, n1)] = status

    with open(file) as f:
        for t in f.readlines():
            tot += 1

            ts = t.split(':')
            r, n2 = ts[0].strip().split()
            op = ts[1].strip()[2:-2].split('), (')

            if (r, n2) == ('angular/angular.js', '14068'):
                continue

            '''
            if r == 'dotnet/corefx':
                # print(r, n2, n1, v, st, sep='\t')
                if 'updat' in (get_pull(r, n2)['title'].lower()):
                    continue
            '''
            
            '''
            if r == 'ceph/ceph':
                continue
            
            
            if r == 'dotnet/corefx':
                continue
            '''
            
            n1, v = op[0].split(',')


            v = float(v)
    
            if v >= threshold:
                p_tot += 1

                if (r, n2, n1) in label:
                    st = label[(r, n2, n1)]
                else:
                    st = 'Unknown'
                    if (n2 in get_another_pull(get_pull(r, n1))) or (n1 in get_another_pull(get_pull(r, n2))):
                        st += '(mention)'

                # print(r, n2, n1, v, st, sep='\t')
                    
                if 'Unknown' in st:
                    # raise Exception('lack label for', r, n2, n1)
                    print('lack label for', r, n2, n1, sep='\t')
                    # print(r, n2, n1, v, st, sep='\t')
                    pass
                
                if 'Y' in st:
                    p_num += 1

                if 'mention' in st:
                    p_num += 1

                p2 = get_pull(r, n2)
                p1 = get_pull(r, n1)
                s2 = '<a href="https://github.com/%s/pull/%s" target="_blank"> %s </a>' % (r, n2, r + '/' + n2 + ':' + p2['title'])
                s1 = '<a href="https://github.com/%s/pull/%s" target="_blank"> %s </a>' % (r, n1, r + '/' + n1 + ':' + p1['title'])
                
                with open(outfile, 'a', encoding="utf-8") as outf:
                    print(s2, '----', s1, '<hr>', file=outf)
    
    
    print(way, 1.0 * p_num / p_tot, p_num, p_tot, tot, sep='\t')
    print('')
    # print(way, 1.0 * p_num / p_tot, sep='\t')


for way in way_select:
    # print('-----%s-----' % way)
    work(way)
'''

work('new')

work('text')
'''